In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [2]:
path ='/Users/fyenne/Downloads/booooks/semester5/R_ML/Mar_NCAA/sb'

In [9]:
tourney_result = pd.read_csv(path+'/train_w_stg2.csv')
test_df = pd.read_csv(path+'/test_w_stg2.csv')

In [22]:
# train_list=list(tourney_result.columns)[:-1]
# tourney_result.columns[-1]

,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,...,X2_OppFgaMean,X2_OppFgaMin,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2,Rating.x,Rating.y,train_label_my
0,0.600000,70.531250,70.5,3.531250,54.937500,54.5,40,77,13.812500,5.343750,...,55.500000,44,7,10,-3,0.916482,0.958527,92.605689,88.227827,win
1,0.750000,81.333333,81.0,14.966667,62.300000,62.5,49,72,16.733333,4.900000,...,64.678571,52,2,1,1,1.066457,1.193799,84.075015,93.380430,win
2,0.333333,80.096774,79.0,8.903226,60.677419,61.0,43,76,16.903226,5.645161,...,64.678571,52,5,1,4,0.982925,1.193799,97.181957,93.380430,loss
3,0.500000,77.064516,73.0,8.870968,55.451613,55.0,35,67,15.741935,3.516129,...,64.678571,52,9,1,8,1.023040,1.193799,84.727965,93.380430,loss
4,1.000000,67.793103,65.0,4.655172,55.862069,58.0,44,67,14.206897,2.965517,...,64.678571,52,16,1,15,0.975313,1.193799,69.892386,93.380430,loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,1.000000,79.030303,80.0,11.939394,58.121212,58.0,47,72,17.090909,4.272727,...,54.147059,40,8,9,-1,1.021250,0.645903,74.666226,82.378832,loss
2226,1.000000,70.485714,72.0,7.542857,56.314286,57.0,40,77,13.342857,4.228571,...,57.545455,47,12,5,7,0.900987,0.960146,87.665786,87.153435,win
2227,0.750000,76.727273,77.0,11.303030,55.606061,56.0,42,65,13.848485,5.060606,...,54.966667,43,2,7,-5,1.427841,0.717883,90.380569,75.599080,win
2228,0.800000,73.939394,73.0,2.454545,59.363636,59.0,45,79,13.545455,3.909091,...,54.966667,43,10,7,3,1.029088,0.717883,83.327530,75.599080,loss


In [23]:
tourney_result = tourney_result.rename(columns = {'train_label_my':'result'})

In [24]:
import h2o
h2o.init(
  nthreads=-1,            ## -1: use all available threads
  max_mem_size = "8G")
train_list=list(tourney_result.columns)[:-1]
train_1=h2o.H2OFrame(tourney_result)
train_1['result']=train_1['result'].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.
Parse progress: |█████████████████████████████████████████████████████████| 100%


H2O_cluster_uptime:,47 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 17 days
H2O_cluster_name:,H2O_from_python_fyenne_2smjm1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.000 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [25]:
param = {
      "ntrees" : 2000
    , "max_depth" : 20
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
    ,"stopping_metric" :"MSE"
    ,"nfolds":8
    ,"fold_assignment":"AUTO"
    ,"keep_cross_validation_predictions" : True
    ,"booster":"dart"
}

In [ ]:
from h2o.estimators import H2OXGBoostEstimator
model_xgb = H2OXGBoostEstimator(**param)
model_xgb.train(x = train_list, y = 'result', training_frame = train_1)
param={
    "ntrees" : 1060
    , "max_depth" : 20
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    , "stopping_metric" :"MSE"
    , "nfolds":8
    , "fold_assignment":"AUTO"
    , "keep_cross_validation_predictions" : True
 }

/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


xgboost Model Build progress: |███████████████████████

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator  # import gbm estimator
model_gbm = H2OGradientBoostingEstimator(**param)
model_gbm.train(x = train_list, y = 'result', training_frame = train_1)
from h2o.estimators import H2OStackedEnsembleEstimator
stack = H2OStackedEnsembleEstimator(model_id="ensemble11",
                                       training_frame=train_1,
                                       #validation_frame=test,
                                       base_models=[model_xgb.model_id,model_gbm.model_id],metalearner_algorithm="glm")
stack.train(x=train_list, y="result", training_frame=train_1)

In [ ]:
test_1=h2o.H2OFrame(test_df)
pred1=model_xgb.predict(test_1)
pred2=model_gbm.predict(test_1)
pred_df1=pred1.as_data_frame()
pred_df2=pred2.as_data_frame()

In [ ]:
ids = pd.read_csv(path + '/WSampleSubmissionStage2.csv').ID.values
values = pred_df2.p1 * 0.7 + pred_df1.p1 * 0.3
# values = pred_df1.p1

final_df = pd.DataFrame(columns=['ID', 'Pred'])
final_df.ID = ids
final_df.Pred = values
final_df.to_csv('submission_women_GBM_worse.csv', columns=['ID','Pred'], index=None)